In [98]:
import scipy.io as scio
import os
import numpy as np
import sklearn.metrics as metrics
# from keras import layers
# from keras import models
import pandas as pd
# from sklearn.metrics import f1_score,recall_score,precision_score,auc,accuracy_score,confusion_matrix
# from sklearn.metrics import precision_recall_curve, average_precision_score,roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import cycle
from scipy import interp
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
import seaborn as sns
from catboost import CatBoostClassifier

In [68]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam,Adadelta
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D,Conv1D,Conv1DTranspose
from tensorflow.keras.layers import MaxPooling1D, Dropout, UpSampling1D,MaxPool2D,UpSampling2D
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# plt.style.use('seaborn')
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import roc_curve, auc, precision_recall_curve
# from tqdm import notebook
import glob
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [69]:
path='./CWRU/'
#寻找文件夹下所有的数据文件
path_list=os.listdir(path)
data_x=[]
data_y=[]
for i in path_list:
    path1=path+i
    dd=scio.loadmat(path1)
    for j in dd.keys():
        #只使用DE的数据
        if 'DE' in j:
            ddd=dd[j]
            #截前12W个数据
            data_x.append(ddd[:120000].ravel())
            data_y.append(i)

In [70]:
#切分成多个样本数据
def yuchuli(a,label):
    """
    前200个样本用于训练
    后100个样本用于测试
    """
    a  = a.reshape(300,400)
    np.random.shuffle(a)
    train = a[:200,:]
    test = a[200:,:]
    label_test =[label for i in range(0,100)]
    label_train = [label for i in range(0, 200)]
    return train,test,label_train ,label_test

    
#将y转变为onehot向量
def to_one_hot(labels,label_dict):
    dimension=len(label_dict.keys())
    results = np.zeros((len(labels),dimension))
    for i,label in enumerate(labels):
        results[i,label_dict[label]] = 1
    return results

In [71]:
#将多条数据的所有样本合并起来
x_train,x_test,y_train,y_test=[],[],[],[]
for i in range(len(data_x)):
    train,test,label_train ,label_test=yuchuli(data_x[i],data_y[i])
    x_train+=list(train)
    x_test+=list(test)
    y_train+=label_train
    y_test+=label_test
    
#转换数据格式
x_train=np.array(x_train)
x_test=np.array(x_test)

#将x转变为二维矩阵，方便CNN训练
x_train = x_train.reshape(-1,400,1)
x_test = x_test.reshape(-1,400,1)

state = np.random.get_state()
np.random.shuffle(x_train)
np.random.set_state(state)
np.random.shuffle(y_train)

#转换y的数据格式
y_train=np.array(y_train)
y_test=np.array(y_test)

#生成label的字典，用于将y转换为onehot格式
label_dict={}
label_dict_anti={}
for i,j in enumerate(list(set(list(y_train)+list(y_test)))):
    label_dict[j]=i
    label_dict_anti[i]=j
    
#将label转化为onehot向量
one_hot_train_labels = to_one_hot(y_train,label_dict)
one_hot_test_labels = to_one_hot(y_test,label_dict)

#将onehot向量转化为标签值
y_test_transform=np.argmax(one_hot_test_labels,axis=1)
y_train_transform=np.argmax(one_hot_train_labels,axis=1)

#获取数据的规模
input_shape=x_train.shape[1:]
output=one_hot_train_labels.shape[1]

In [72]:
one_hot_train_labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [73]:
x_train.shape,x_test.shape

((2000, 400, 1), (1000, 400, 1))

In [74]:
y_train_transform.shape,y_test_transform.shape

((2000,), (1000,))

Model

In [75]:
num_classes = len(np.unique(y_train_transform))
num_classes

10

In [76]:
# INPUT_DIMS = 40
# TIME_STEPS = 400
# lstm_units = 64
import keras.backend as K
from keras.layers import Input, Dense, LSTM, merge ,Conv1D,Dropout,Bidirectional,Multiply
from keras.models import Model
from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

import  pandas as pd
import  numpy as np

In [77]:
SINGLE_ATTENTION_VECTOR = False
def attention_3d_block2(inputs, single_attention_vector=False):
    # 如果上一层是LSTM，需要return_sequences=True
    # inputs.shape = (batch_size, time_steps, input_dim)
    time_steps = K.int_shape(inputs)[1]
    input_dim = K.int_shape(inputs)[2]
    a = inputs
    a = Dense(input_dim, activation='softmax')(a)
    if single_attention_vector:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)

    a_probs = Permute((1, 2))(a)
    # 乘上了attention权重，但是并没有求和，好像影响不大
    # 如果分类任务，进行Flatten展开就可以了
    # element-wise
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

In [95]:
def attention_model():
    input_img = Input(shape=(400, 1))

    x= Conv1D(filters = 30, kernel_size = 10, activation = 'relu', padding = 'same')(input_img)
    # x = layers.LSTM(units=30,return_sequences=True)(x)
    # x = Flatten()(x)

    y= Conv1D(filters = 30, kernel_size = 20, activation = 'relu', padding = 'same')(input_img)
    # y = layers.LSTM(units=30,return_sequences=True)(y)
    # y = Flatten()(y)

    z= Conv1D(filters = 30, kernel_size = 30, activation = 'relu', padding = 'same')(input_img)
    # z = layers.LSTM(units=30,return_sequences=True)(z)
    # z = Flatten()(z)

    inception_out=tf.concat([x,y,z],axis=-1)
        
    
    attention_mul = attention_3d_block2(inception_out)
    #lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
    # lstm_out = LSTM(64, return_sequences=True)(attention_mul)


    # attention_mul = attention_3d_block2(inception_out)
    attention_mul = Flatten()(attention_mul)

    output = Dense(10, activation='softmax')(attention_mul)
    model = Model(inputs=input_img, outputs=output)
    return model

In [99]:
model=attention_model()

In [100]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])   

In [81]:
#保存模型参数，用于提取输出特征
model.save_weights('./CNN_weight.pkl copy')

In [101]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 400, 1)]     0           []                               
                                                                                                  
 conv1d_12 (Conv1D)             (None, 400, 30)      330         ['input_5[0][0]']                
                                                                                                  
 conv1d_13 (Conv1D)             (None, 400, 30)      630         ['input_5[0][0]']                
                                                                                                  
 conv1d_14 (Conv1D)             (None, 400, 30)      930         ['input_5[0][0]']                
                                                                                            

In [83]:
x_temp1=model.predict(x_train)

In [84]:
x_temp2=model.predict(x_test)

In [85]:
x_temp1.shape,x_temp2.shape

((2000, 10), (1000, 10))

In [106]:
model = CatBoostClassifier(iterations=500,learning_rate=0.01,eval_metric='AUC',random_seed=42,verbose=True,task_type='CPU')
model.fit(x_temp1, y_train_transform)
svm_line_pred_test=model.predict_proba(x_temp2)
# svm_line_pred_test_1=np.argmax(svm_line_pred_test,axis=1)

0:	learn: 2.2940235	total: 48.9ms	remaining: 196ms
1:	learn: 2.2842795	total: 86.8ms	remaining: 130ms
2:	learn: 2.2762529	total: 128ms	remaining: 85.5ms
3:	learn: 2.2677735	total: 170ms	remaining: 42.6ms
4:	learn: 2.2587216	total: 225ms	remaining: 0us


In [90]:
svm_line_pred_test.shape

(1000, 10)

In [91]:
pre=np.argmax(svm_line_pred_test,axis=1)

In [92]:
test_y=np.argmax(one_hot_test_labels,axis=1)

In [103]:
correct_prediction = tf.equal(pre, test_y)
acc = tf.reduce_mean(tf.cast(correct_prediction, "float"))
acc=float(acc)
# f1=metrics.f1_

In [104]:
acc

0.5509999990463257